# 컴퓨터 파일에서 이미지를 받아와서 좌클릭을하면 색상 코드 등록, 엑셀에 저장하고 싶으면 아래 버튼을 클릭

In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import openpyxl
import sys

# 엑셀 파일 초기화
def initialize_excel():
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Colors"
    ws.append(["X", "Y", "HEX Color"])
    return wb, ws

# 클릭한 좌표의 색상을 가져와 엑셀에 저장
def on_click(event):
    global img, ws
    if img:  # 이미지가 로드되었는지 확인
        x, y = event.x, event.y
        try:
            color = img.getpixel((x, y))
            hex_color = '#{:02x}{:02x}{:02x}'.format(*color)
            ws.append([x, y, hex_color])
            print(f"Coordinates: ({x}, {y}) - HEX Color: {hex_color}")
        except IndexError:
            print("Clicked outside the image boundaries.")
    else:
        print("No image loaded. Please open an image first.")

# 파일 열기 및 이미지 표시
def open_image():
    global img, tk_img
    file_path = filedialog.askopenfilename(
        filetypes=[("Image Files", "*.png *.jpg *.jpeg *.bmp *.gif")]
    )
    if file_path:
        img = Image.open(file_path).convert("RGB")  # 이미지를 RGB로 변환
        tk_img = ImageTk.PhotoImage(img)
        canvas.config(scrollregion=(0, 0, img.width, img.height))  # 스크롤 영역 설정
        canvas.create_image(0, 0, anchor=tk.NW, image=tk_img)  # 이미지 표시
        print(f"Loaded image: {file_path}")

# 엑셀 저장
def save_excel():
    file_path = filedialog.asksaveasfilename(
        defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")]
    )
    if file_path:
        wb.save(file_path)
        print(f"Excel file saved at: {file_path}")

# 프로그램 종료
def exit_program():
    print("Exiting program.")
    try:
        if wb:
            wb.close()  # 엑셀 파일 닫기
    except Exception as e:
        print(f"Error closing workbook: {e}")
    
    root.quit()    # Tkinter 메인 루프 종료
    root.destroy() # Tkinter 창 제거




# GUI 설정
root = tk.Tk()
root.title("Image Color Picker")

# 메인 프레임 생성
main_frame = tk.Frame(root)
main_frame.pack(fill=tk.BOTH, expand=True)

# 캔버스 프레임 생성 (스크롤바 포함)
canvas_frame = tk.Frame(main_frame)
canvas_frame.pack(fill=tk.BOTH, expand=True)

# 캔버스 생성
canvas = tk.Canvas(canvas_frame, bg="gray")
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 스크롤바 추가
scroll_x = tk.Scrollbar(canvas_frame, orient=tk.HORIZONTAL, command=canvas.xview)
scroll_x.pack(side=tk.BOTTOM, fill=tk.X)
scroll_y = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL, command=canvas.yview)
scroll_y.pack(side=tk.RIGHT, fill=tk.Y)

canvas.configure(xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)

# 이미지 초기화
img = None  # 이미지 로드 여부 확인용 변수
tk_img = None

# 버튼 프레임 생성
button_frame = tk.Frame(main_frame)
button_frame.pack(fill=tk.X, pady=5)

open_button = tk.Button(button_frame, text="Open Image", command=open_image)
open_button.pack(side=tk.LEFT, padx=5)

save_button = tk.Button(button_frame, text="Save to Excel", command=save_excel)
save_button.pack(side=tk.LEFT, padx=5)

exit_button = tk.Button(button_frame, text="Exit", command=exit_program)
exit_button.pack(side=tk.RIGHT, padx=5)

# 이벤트 바인딩
canvas.bind("<Button-1>", on_click)

# 엑셀 초기화
wb, ws = initialize_excel()

root.mainloop()


2025-02-14 17:41:36.251 Python[34698:1830223] +[IMKClient subclass]: chose IMKClient_Modern


Loaded image: /Users/parkjiyon/Desktop/취업/블로그/벚꽃.jpeg


# 엑셀 파일명만 바꿔주면 색상 코드가 있는 곳 배경을 그 색으로 칠해줌

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

def apply_colors_to_excel(file_path, sheet_name="Colors", color_column="C"):
    """
    엑셀 파일에서 HEX 색상 코드를 읽고 해당 셀의 배경색을 설정합니다.

    Args:
        file_path (str): 엑셀 파일 경로
        sheet_name (str): 색상 코드가 기록된 시트 이름
        color_column (str): HEX 색상 코드가 기록된 열 (기본값: C열)
    """
    try:
        # 엑셀 파일 로드
        wb = load_workbook(file_path)
        ws = wb[sheet_name]

        # C열에 기록된 HEX 색상 코드를 읽어서 배경색 설정
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=3, max_col=3):
            for cell in row:
                hex_color = cell.value
                if hex_color and hex_color.startswith("#") and len(hex_color) == 7:
                    # 배경색 설정
                    fill = PatternFill(start_color=hex_color[1:], end_color=hex_color[1:], fill_type="solid")
                    cell.fill = fill

        # 수정된 엑셀 저장
        wb.save(file_path)
        print(f"Colors applied successfully to the file: {file_path}")

    except Exception as e:
        print(f"Error: {e}")

# 사용 예시
apply_colors_to_excel("sakuratest.xlsx", sheet_name="Colors", color_column="C")
